# GENERATE A BACKEND PROJECT 

+ this notebook is a template to generate Backend-required files to view a project with the DataDiVR (preview or VR)
+ STEP 1 and the "create a graph" section contains a template graph writing a required format (json) to then use the generate-project functions of the DataDiVR backend

+ STEP 2 to actually generate BACKEND project files.

In [6]:
import networkx as nx
import json 
import os

# these are the two functions one needs to create a JSON file to upload and create the project in the backend 
import nx2json as nx2j 
from uploaderGraph import upload_filesJSON

# some background knowledge: 

### THE GRAPH STRUCTURE FORMAT generated in this notebook to create a Porject on the backend 

In [3]:
'''

{
----------------------------------------
THIS IS THE GENERAL GRAPH INFO SECTION
----------------------------------------
  "directed": false,
  "multigraph": false,
  "projectname": "Testgraph",
  "info": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",
  "graphlayouts": [
      "layout1-spring",
      "layout2-spring",
      "layout3-spring",
      "layout4-clusters"
  ],
  "annotationTypes": true,
  "nodes": [
   ----------------------------------------
   contains all nodes of the project
   ----------------------------------------
      {
          "id": 0,
          "name": 0,
          "annotation": 
                {
                    "annot1": [
                        "lambda",
                        "alpha",
                        "zeta",
                        "theta"
                    ],
                    "annot2": [
                        "delta",
                        "nu"
                    ],
                    "annot3": [
                        "mu",
                        "gamma"
                    ]
          }
      },....
  ],
  "links": [
   ----------------------------------------
   contains all links of the project
   ----------------------------------------
      {
          "id": 0,
          "source": 0,
          "target": 1
      },
      {
          "id": 1,
          "source": 0,
          "target": 2
      },...
       ],
  "layouts": [
   ----------------------------------------
   contains all layouts of the project
   only contains nodes and links as well as colors specific to the layout
   ----------------------------------------
       {  "layoutname" : "name of first layout",
          "nodes": [
              {
                  "nodecolor": [
                      255,
                      35,
                      0,
                      120
                  ],
                  "pos": [
                      -0.5618057865250979,
                      0.1467411221839164,
                      0.49656801102094605
                  ],
                  "id": 0
               },...
        	],
          "links": [
              {
                  "linkcolor": [
                      0,
                      255,
                      0,
                      100
                  ],
                  "source": 0,
                  "target": 1
              },...
         	],
   	  }, {
          "layoutname" : "name of second layout",
          "nodes": [
              {
                  "nodecolor": "#0000ffaa",
                  "pos": [
                      -0.35948900932978317,
                      0.6255258442839948,
                      -0.04209289102217994
                  ],
                  "cluster": "cluster group 1",
                  "id": 0
               },... 
],
          "links": [
              {
                  "linkcolor": "#0000ff",
                  "source": 0,
                  "target": 1
              },
],
  	   }, { . . .  
 	},
}

'''

'\n\n{\n----------------------------------------\nTHIS IS THE GENERAL GRAPH INFO SECTION\n----------------------------------------\n  "directed": false,\n  "multigraph": false,\n  "projectname": "Testgraph",\n  "info": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",\n  "graphlayouts": [\n      "layout1-spring",\n      "layout2-spring",\n      "layout3-spring",\n      "layout4-clusters"\n  ],\n  "annotationTypes": true,\n  "nodes": [\n   ----------------------------------------\n   contains all nodes of the project\n   ----------------------------------------\n      {\n          "id": 0,\n          "name": 0,\n          "annotation": \n                {\n                    "annot1": [\n                        "lambda",\n                        "alpha",\n                        "zeta",\n                        "theta"\n                    ],\n                    "annot2": [\n                        "delta",\n                        "nu"\n                    ],\n    

## **How this is meant to be used:**
+ Create an nx.Graph with stored node positions ("pos") and colors ("nodecolor") and link colors ("linkcolor")
+ set attributes in the nx.Graph accordingly
+ then merge all layouts (nx.Graphs) using the "merge_graphs" function to create one large file, containing all information for your project 
+ this will generate the file structure stated above 

# STEP 1a - Create Graphs with stored positions, colors etc. or use your own nx.Graphs

### nx.Graph

In [18]:
G = nx.random_geometric_graph(100,0.2)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================

#########################################################################################
G.name = "testproject" # choose name of the project 
#########################################################################################

G.graph['projectname'] = G.name
G.graph['info'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  100
Number of Links:  544


### create node anntotations

In [19]:
import random 
def generate_random_words(num):
    words = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta", "theta", "lambda", "mu", "nu"]
    return random.sample(words, num)

# Create a list to hold annotations in JSON format
l_annotations_json = []

# Process each node in the graph
for g in G.nodes():
    # Generate random annotations
    annotations = {
        "annot1": generate_random_words(random.randint(3, 4)),
        "annot2": generate_random_words(random.randint(1, 2)),
        "annot3": generate_random_words(random.randint(2, 3))
    }
    
    l_annotations_json.append(annotations)

# Create a dictionary mapping nodes to their annotations
d_annotations = dict(zip(G.nodes(), l_annotations_json))

# Set the node attributes in the graph
nx.set_node_attributes(G, d_annotations, name="annotation")


### create node positions and set as "pos" Graph attribute 
here are 3 different layouts, which all are stored in unique nx.Graph-objects (G_rgba, G_hex, ....)

In [11]:
# First layout (i.e. Graph 1)
G_rgba = G.copy()
G_rgba.name = 'layout1-spring'
G_rgba.graph["scene"] = G_rgba.name
posG3D_1_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=100)
posG3D_1 = {key: value.tolist() for key, value in posG3D_1_pre.items()}
nx.set_node_attributes(G_rgba, posG3D_1, name="pos")


# Second layout (i.e. Graph 2)
G_hex = G.copy()
G_hex.name = 'layout2-spring'
G_hex.graph["scene"] = G_hex.name
posG3D_2_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=200)
posG3D_2= {key: value.tolist() for key, value in posG3D_2_pre.items()}
nx.set_node_attributes(G_hex, posG3D_2, name="pos")


# Third layout (i.e. Graph 3)
G_hex8 = G.copy()
G_hex8.name = 'layout3-spring'
G_hex8.graph["scene"] = G_hex8.name
posG3D_3_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=500)
posG3D_3 = {key: value.tolist() for key, value in posG3D_3_pre.items()}
nx.set_node_attributes(G_hex8, posG3D_3, name="pos")


# Fourth layout (i.e. Graph 4) - with clusters
G_clusters = G.copy()
G_clusters.name = 'layout4-clusters'
G_clusters.graph["scene"] = G_clusters.name
clustername_1 = 'cluster group 1'
clustername_2 = 'cluster group 2'
clustername_3 = 'cluster group 3'

# nodes into groups
for g in G_clusters.nodes():
    if g < len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_1
    elif g < 2 * len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_2
    else:
        G_clusters.nodes[g]['cluster'] = clustername_3

nx.set_node_attributes(G_clusters, posG3D_2, name="pos")

#### node and link colors 

In [12]:
# 3 Formats of colors values are supported: hex, rgba, hex8

# FIRST GRAPH - rgba color values
d_nodecolors_rgba = dict(zip(G_rgba.nodes(),[(255,35,0,120)]*len(G_rgba.nodes())))
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
l_linkcolors_rgba = (0,255,0,100)
nx.set_edge_attributes(G_rgba, l_linkcolors_rgba, name="linkcolor")


# SECOND GRAPH - hex color values 
d_nodecolors_hex = dict(zip(G_hex.nodes(),['#FF2300']*len(G_hex.nodes())))
nx.set_node_attributes(G_hex, d_nodecolors_hex, name="nodecolor")
l_linkcolors_hex = '#ff0000'
nx.set_edge_attributes(G_hex, l_linkcolors_hex, name="linkcolor")


# THIRD GRAPH - hex8 color values
d_nodecolors_hex8 = dict(zip(G_hex8.nodes(),['#0000ffaa']*len(G_hex8.nodes())))
nx.set_node_attributes(G_hex8, d_nodecolors_hex8, name="nodecolor")
l_linkcolors_hex8 = '#0080ffaa'
nx.set_edge_attributes(G_hex8, l_linkcolors_hex8, name="linkcolor")


# FOURTH GRAPH - clusters assigned 

# node colors 
d_nodecolors_clusters = {}
nodes_group1 = []
nodes_group2 = []
nodes_group3 = []
for n in G_clusters.nodes(): 
    if G_clusters.nodes[n]['cluster'] == clustername_1:
        d_nodecolors_clusters[n] = '#0000ffaa'
        nodes_group1.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_2:
        d_nodecolors_clusters[n] = '#00ff00aa'
        nodes_group2.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_3:
        d_nodecolors_clusters[n] = '#ff0000aa'
        nodes_group3.append(n)

# link colors
d_linkcolors_clusters = {}
for edge in G_clusters.edges():
    if edge[0] in nodes_group1 and edge[1] in nodes_group1:
        d_linkcolors_clusters[edge] = '#0000ff'
       
    elif edge[0] in nodes_group2 and edge[1] in nodes_group2:
        d_linkcolors_clusters[edge] = '#00ff00'
       
    elif edge[0] in nodes_group3 and edge[1] in nodes_group3:
        d_linkcolors_clusters[edge] = '#ff0000'
       
    else:
        d_linkcolors_clusters[edge] = '#B1B1B1'

l_linkcolors_clusters = list(d_linkcolors_clusters.values())

nx.set_node_attributes(G_clusters, d_nodecolors_clusters, name="nodecolor")
nx.set_edge_attributes(G_clusters, {edge: color for edge, color in zip(G_clusters.edges(), l_linkcolors_clusters)}, "linkcolor")


In [ ]:
# TO DO 
# test visualization in case of graph edges different in count 


### merge Graph files into one and store as a json file

In [13]:
Graphs = [G_rgba, G_hex, G_hex8, G_clusters]

# STEP 1b: use your own nx.Graph objects, store them in a list 

In [14]:
from networkx.readwrite import json_graph

tempfolder = "temp-json"
if not os.path.exists(tempfolder):
    os.makedirs(tempfolder)

# for G in Graphs:
#     G_serializable = json_graph.node_link_data(G) # ensure json serializability
#     with open(os.path.join(tempfolder,G.name + '.json'), 'w') as f:
#         json.dump(G_serializable, f, indent=4)

In [15]:
# read json files from temp folder into a list of nx.Graph objects 
Graphs = []
for file in os.listdir(tempfolder):
    if file.endswith(".json"):
        with open(os.path.join(tempfolder, file), 'r') as f:
            data = json.load(f)
            G = json_graph.node_link_graph(data)
            Graphs.append(G)

Graphs

# THE ACTUAL FUNCTION TO GENERATE THE JSON FILE THE VR BACKEND REQUIRES

In [16]:
# then use merge_graphs to get the actual format for the VR backend 
G_merged = nx2j.merge_graphs(Graphs)

Merged JSON file saved as:  /Users/chris/Desktop/Github/DataDiVR_WebApp/MYPROJECT_1.json


# STEP 2 - CREATE PROJECT - for the VR BACKEND

In [17]:
upload_filesJSON(G_merged)

Successfully created the directory static/projects/MYPROJECT_1 
C_DEBUG: file in linkcolors:  {'layoutname': 'layout2-spring', 'nodes': [{'nodecolor': '#FF2300', 'pos': [-0.13959908253224984, -0.2914696347091415, 0.536443375426691], 'cluster': None, 'id': 0}, {'nodecolor': '#FF2300', 'pos': [0.5886931363277835, 0.2520960484331904, -0.9499306150772382], 'cluster': None, 'id': 1}, {'nodecolor': '#FF2300', 'pos': [0.3516452455778018, 0.3677469854294571, -0.652081292557149], 'cluster': None, 'id': 2}, {'nodecolor': '#FF2300', 'pos': [0.32417438510089347, -0.534929538556518, 0.32792021448940983], 'cluster': None, 'id': 3}, {'nodecolor': '#FF2300', 'pos': [0.14066056056706946, 0.10166814008541192, -0.37266762054552016], 'cluster': None, 'id': 4}, {'nodecolor': '#FF2300', 'pos': [-0.1714915349901331, 0.4015743676236535, -0.08396827376412294], 'cluster': None, 'id': 5}, {'nodecolor': '#FF2300', 'pos': [0.4157606319945667, -0.25312023792611393, -0.09877264298951159], 'cluster': None, 'id': 6}, 

'<a style="color:green;">SUCCESS </a>layout2-spring NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>layout1-spring NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>Layoutname2 NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>layout3-spring NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>layout4-clusters NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>layout2-spring Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>layout1-spring Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>Layoutname2 Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>layout3-spring Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>layout4-clusters Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>layout2-spring Link Textures Created<br><a style="color:green;">SUCCESS </a>layout1-spring Link Textures Created<br><a style="color:green;">SUCCESS </a>Layoutname2 L

# OR: read merged json (with keys "nodes", "links" and "layouts") and upload 

In [2]:
# ----------------------------------------
# DEFINE filename HERE : 
filename = 'NDIM_Sphere.json'
# ----------------------------------------
currentwd = '/Users/chris/Desktop/Github/DataDiVR_Webapp/temp-json/'

In [3]:
# ----------------------------------------
# DEFINE filename HERE : 
#filename = 'GOMF.json'
#filename = "GI_net.json"
# ----------------------------------------
#currentwd = '/Users/chris/Desktop/_Homie_Networks/Mathilde/convert_graphs_to_VR_Mathilde/'

In [4]:
path = os.path.join(currentwd, filename)

with open(path, 'r') as f:
     G_merged = json.load(f)

In [5]:
upload_filesJSON(G_merged)

Successfully created the directory static/projects/NDIM_Sphere 
C_DEBUG: file in linkcolors:  {'layoutname': '00-Sphere-volume_3D_200nodes', 'nodes': [{'nodecolor': '#bad0f8', 'pos': [0.7064009008465858, 0.5127378934993668, 0.83348191826842], 'cluster': None, 'id': 0}, {'nodecolor': '#f5c4ac', 'pos': [0.4530132534638411, 0.22996024886132455, 0.6318077117149947], 'cluster': None, 'id': 1}, {'nodecolor': '#eed0c0', 'pos': [0.56415811140716, 0.36284757041604415, 0.21042665351000137], 'cluster': None, 'id': 2}, {'nodecolor': '#6b8df0', 'pos': [0.45410207696702415, 0.9713028638571356, 0.3623560751837738], 'cluster': None, 'id': 3}, {'nodecolor': '#f2cab5', 'pos': [0.5549046886335187, 0.40981066801043636, 0.20731025414923643], 'cluster': None, 'id': 4}, {'nodecolor': '#9abbff', 'pos': [0.5505505404780902, 0.2231504084925658, 0.8597275685240707], 'cluster': None, 'id': 5}, {'nodecolor': '#86a9fc', 'pos': [0.14113178096720316, 0.33021487587388143, 0.20481291633927262], 'cluster': None, 'id': 6

'<a style="color:green;">SUCCESS </a>00-Sphere-volume_3D_200nodes NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>01-Sphere-volume_4D_200nodes NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>02-Sphere-volume_5D_200nodes NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>03-Sphere-volume_6D_200nodes NodeXYZ Textures Created<br><a style="color:green;">SUCCESS </a>00-Sphere-volume_3D_200nodes Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>01-Sphere-volume_4D_200nodes Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>02-Sphere-volume_5D_200nodes Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>03-Sphere-volume_6D_200nodes Nodecolor Textures Created<br><a style="color:green;">SUCCESS </a>00-Sphere-volume_3D_200nodes Link Textures Created<br><a style="color:green;">SUCCESS </a>01-Sphere-volume_4D_200nodes Link Textures Created<br><a style="color:green;">SUCCESS </a>02-Sphere-volume_5D_200nodes Li